# PyTorch Linear Layers & ResNets Guide

A beginner's guide to building neural networks with PyTorch

## Overview

This notebook provides a practical guide to building neural networks with PyTorch, focusing on residual architectures and data handling pipelines.

### Concepts Covered:

1. **Imports** - Essential PyTorch libraries for building neural networks and handling data.
2. **ResBlock (Module Class)** - A residual block with skip connections built using individual layer definitions.
3. **ResBlock (Sequential)** - The same residual architecture implemented using `torch.nn.Sequential` for cleaner code.
4. **ResNet** - A complete residual network combining multiple ResBlocks with nested skip connections.
5. **Custom Dataset** - Creating a custom PyTorch Dataset class to generate and manage training data.
6. **DataLoader** - Efficiently batching and loading data for training with shuffling and parallel workers.
7. **Custom Sampler** - Implementing a custom sampling strategy to control how data is accessed during training.
8. **Model Persistence** - Saving trained models to disk and loading them for inference or continued training.
9. **Reproducibility** - Setting random seeds and deterministic algorithms to ensure consistent results across runs.

## 1. Imports

In [1]:
import numpy
import torch
from torch.utils.data import Dataset, DataLoader, Sampler

## 2. ResBlock Using Module Class

Building a residual block with skip connections

In [2]:
class ResBlock(torch.nn.Module):
    """Residual Block with skip connection"""
    
    def __init__(self, input_size=50, hiddens=[128, 32]):
        super(ResBlock, self).__init__()
        self.input_size = input_size
        self.hiddens = hiddens
        
        # Layers
        self.ll1 = torch.nn.Linear(self.input_size, hiddens[0], bias=True)
        self.leaky_relu = torch.nn.LeakyReLU(negative_slope=0.2)
        self.ll2 = torch.nn.Linear(self.hiddens[0], hiddens[1], bias=True)
        self.sigmoid = torch.nn.Sigmoid()
        self.ll3 = torch.nn.Linear(self.hiddens[1], input_size, bias=True)
        self.batch_norm = torch.nn.BatchNorm1d(input_size)
        
    def forward(self, x):
        residual = x
        x = self.ll1(x)
        x = self.leaky_relu(x)
        x = self.ll2(x)
        x = self.sigmoid(x)
        x = self.ll3(x)
        x += residual  # Skip connection
        x = self.batch_norm(x)
        return x

In [3]:
# Initialize ResBlock
resblock1 = ResBlock(20, [10, 5])
print(resblock1)

ResBlock(
  (ll1): Linear(in_features=20, out_features=10, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.2)
  (ll2): Linear(in_features=10, out_features=5, bias=True)
  (sigmoid): Sigmoid()
  (ll3): Linear(in_features=5, out_features=20, bias=True)
  (batch_norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [4]:
# Test ResBlock
data = torch.rand(100, 20)
output = resblock1(data)
print(f"Output shape: {output.shape}")  # Should be [100, 20]

Output shape: torch.Size([100, 20])


## 3. ResBlock Using Sequential

Same architecture using `torch.nn.Sequential`

In [5]:
class ResBlock2(torch.nn.Module):
    """ResBlock using Sequential"""
    
    def __init__(self, input_size=50, hiddens=[128, 32]):
        super(ResBlock2, self).__init__()
        self.input_size = input_size
        
        self.seq = torch.nn.Sequential(
            torch.nn.Linear(self.input_size, hiddens[0], bias=True),
            torch.nn.LeakyReLU(negative_slope=0.2),
            torch.nn.Linear(hiddens[0], hiddens[1], bias=True),
            torch.nn.Sigmoid(),
            torch.nn.Linear(hiddens[1], input_size, bias=True)
        )
        self.batch_norm = torch.nn.BatchNorm1d(input_size)
        
    def forward(self, x):
        residual = x
        x = self.seq(x)
        x += residual
        x = self.batch_norm(x)
        return x

In [6]:
# Test ResBlock2
resblock2 = ResBlock2(20, [10, 5])
output2 = resblock2(data)
print(f"Output shape: {output2.shape}")

Output shape: torch.Size([100, 20])


## 4. ResNet - Combining Multiple ResBlocks

Architecture: `F(F(x)) + F(x) + x`

In [7]:
class ResNet(torch.nn.Module):
    """ResNet with two ResBlocks and multiple skip connections"""
    
    def __init__(self, input_size=50, hiddens=[128, 64]):
        super(ResNet, self).__init__()
        self.resblock1 = ResBlock(input_size, hiddens)
        self.resblock2 = ResBlock2(input_size, hiddens)
        
    def forward(self, x):
        x1 = self.resblock1(x)      # F(x)
        x2 = self.resblock2(x1)      # F(F(x))
        output = x + x1 + x2         # x + F(x) + F(F(x))
        return output

In [8]:
# Initialize and test ResNet
nnet = ResNet(20, [10, 5])
random_data = torch.rand(100, 20)
output_res = nnet(random_data)
print(f"Output shape: {output_res.shape}")

Output shape: torch.Size([100, 20])


## 5. Custom Dataset

In [9]:
class RandomDataset(Dataset):
    """Generate random data with binary labels"""
    
    def __init__(self, num_samples=128, feature_dim=50):
        self.len = num_samples
        self.data = torch.randn(self.len, feature_dim)
        self.labels = torch.randint(0, 2, (self.len,))

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [10]:
# Test dataset
dataset = RandomDataset()
sample_data, sample_label = dataset[0]
print(f"Data shape: {sample_data.shape}")
print(f"Label: {sample_label}")

Data shape: torch.Size([50])
Label: 0


## 6. DataLoader

In [11]:
# Create DataLoader
training_loader = DataLoader(
    dataset,
    batch_size=8,
    shuffle=True,
    drop_last=True,
    num_workers=0
)

print(f"Number of batches: {len(training_loader)}")

Number of batches: 16


In [12]:
# Test DataLoader
for batch_idx, (data, target) in enumerate(training_loader):
    print(f"Batch {batch_idx}: data shape = {data.shape}, target shape = {target.shape}")
    if batch_idx == 2:
        break

Batch 0: data shape = torch.Size([8, 50]), target shape = torch.Size([8])
Batch 1: data shape = torch.Size([8, 50]), target shape = torch.Size([8])
Batch 2: data shape = torch.Size([8, 50]), target shape = torch.Size([8])


## 7. Custom Sampler

In [13]:
class RandomSampler(torch.utils.data.Sampler):
    """Random sampler for dataset"""
    
    def __init__(self, num_samples=128):
        self.num_samples = num_samples
        
    def __iter__(self):
        indices = torch.randperm(self.num_samples).tolist()
        return iter(indices)
    
    def __len__(self):
        return self.num_samples

In [14]:
# Use custom sampler
sampler = RandomSampler(128)
loader_with_sampler = DataLoader(
    dataset,
    batch_size=16,
    sampler=sampler,
    num_workers=0
)

## 8. Saving and Loading Models

In [15]:
# Save model
torch.save(nnet.state_dict(), 'resnet_model.pth')
print("Model saved!")

# Load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loaded_model = ResNet(input_size=20, hiddens=[10, 5])
loaded_model.load_state_dict(torch.load('resnet_model.pth', map_location=device))
loaded_model.to(device)
loaded_model.eval()
print(f"Model loaded on {device}!")

# Test loaded model
test_data = torch.rand(10, 20).to(device)
with torch.no_grad():
    output = loaded_model(test_data)
print(f"Test output shape: {output.shape}")

Model saved!
Model loaded on cpu!
Test output shape: torch.Size([10, 20])


## 9. Reproducibility

In [16]:
# Set random seeds for reproducibility
import random

numpy.random.seed(42)
random.seed(42)
torch.manual_seed(42)

# For deterministic algorithms
torch.use_deterministic_algorithms(True)

print("Reproducibility settings applied!")

Reproducibility settings applied!
